<a href="https://colab.research.google.com/github/suinkangme/Word2Vec/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Experiments with Word Embeddings

In [1]:
# import libraries
import pandas as pd
import numpy as np
import gensim
import warnings

### Import the MyDrive library from CoLab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# the directory owns this notebook and dataset
import os
os.chdir('/content/drive/MyDrive/comp472/')

Mounted at /content/drive


### Load the dataset

In [3]:
# load dataset
data_synonym = pd.read_csv('synonym.csv')
data_synonym.head()

,question,answer,0,1,2,3
0,enormously,tremendously,appropriately,uniquely,tremendously,decidedly
1,provisions,stipulations,stipulations,interrelations,jurisdictions,interpretations
2,haphazardly,randomly,dangerously,densely,randomly,linearly
3,prominent,conspicuous,battered,ancient,mysterious,conspicuous
4,zenith,pinnacle,completion,pinnacle,outset,decline


## Task 1

In [4]:
import gensim.downloader as api

info = api.info()
model = api.load("word2vec-google-news-300")

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [5]:
# create output files 'word2vec-google-news-300-details.csv'
outputFile = open('word2vec-google-news-300-details.csv', 'w')
warnings.filterwarnings('ignore')

outputFile.write("question_word,correct_answer,system_guess,label\n")

48

In [6]:
# define and extract word from data_synonym
question_word = data_synonym.iloc[:,0].values
correct_word = data_synonym.iloc[:,1].values
guess_word_0 = data_synonym.iloc[:,2].values
guess_word_1 = data_synonym.iloc[:,3].values
guess_word_2 = data_synonym.iloc[:,4].values
guess_word_3 = data_synonym.iloc[:,5].values

In [7]:
for i in range(len(question_word)):

    # the question word is found in the model
    if question_word in model.vocab:

      # compute the cosine similarity between the two vectors
      # if the word is matched at least one, label 'correct'
      if any(word in model.vocab for word in [guess_word_0, guess_word_1, guess_word_2, guess_word_3]):

          sim_scores = [
              model.similarity(question_word, guess_word)
              for guess_word in [guess_word_0, guess_word_1, guess_word_2, guess_word_3]
              if guess_word in model.vocab
              ]

          # find the max index of sim_score and get the word
          max_index = sim_scores.index(max(sim_scores))
          system_guess = [guess_word_0, guess_word_1, guess_word_2, guess_word_3][max_index]

          # check if the system's guess is correct
          if system_guess == correct_word:
              label = 'correct'
          else:
              label = 'wrong'

      # all four guess-words were not found in the model
      else:
            system_guess = ''
            label = 'guess'

    # the question word is NOT found in the model
    else:
        system_guess = ''
        label = 'guess'

    outputFile.write(f"{question_word},{correct_word},{system_guess},{label}\n")

outputFile.close()

AttributeError: ignored

In [ ]:
# create output files 'analysis.csv,'
outputFile = open('analysis.csv,', 'w')
warnings.filterwarnings('ignore')

## Task 2

## Task 3